In [ ]:
#!pip install boto3

In [ ]:
import json
import requests
import time
import boto3
import os
import shutil
import telegram

### S3 amazon aws clients

In [ ]:
s3 = boto3.resource(
    service_name='s3',
    region_name='us-east-2',
    aws_access_key_id='your_acces_key_id',
    aws_secret_access_key='your_secret_acces_key'
)

s3_client = boto3.client(service_name='s3',
    region_name='us-east-2',
    aws_access_key_id='your_acces_key_id',
    aws_secret_access_key='your_secret_acces_key')

### Instagram Access Tokens

In [ ]:
access_token = 'your_access_token'
client_id = 'your_client_id'
client_secret = 'your_client_secret'
ig_user_id = 'ig_user_id'

### List bucket in S3

In [ ]:
#List buckets and get only "videos-thisischileofc"
for bucket in s3.buckets.all():
    if 'videos-thisischileofc' in bucket.name:
        bucket_name = bucket.name
        #print(bucket.name)

### Upload process

In [ ]:
class upload_videos:
    
    def __init__(self):
        
        #List mp4 videos on windows folder
        
        self.source_videos = 'your_videos_path'
        #self.source_videos='.'
        self.files = []
        for file in os.listdir(self.source_videos):
            if file.endswith('.mp4'):
                self.files.append(file)
        
        self.files.sort()
        self.name_media = self.files[0]
        self.title = self.name_media[0:-4]
        self.source_last_video = self.source_videos+'/{}'.format(self.name_media)
        

    def upload_instagram(self):

        #Upload video to s3 bucket
        s3.Bucket(bucket_name).upload_file(Filename=self.source_last_video, Key=self.name_media)

        #Generate Public URL of the video 
        video_url = s3_client.generate_presigned_url('get_object',Params={'Bucket': bucket_name,'Key': self.name_media},ExpiresIn=600)
        #video_url

        #upload process video to instagram
        post_url = 'https://graph.facebook.com/v15.0/{}/media'.format(ig_user_id)

        payload={'media_type':'REELS', 
                 'video_url':video_url,
                 'caption':self.title+' #thisischile #solopasaenchile #solochile #chile #chilegram #instachile #chilememes',
                 'access_token' : access_token,
                 'share_to_feed':'true'}

        r1 = requests.post(post_url, data = payload)
        #print(r.text)

        results = json.loads(r1.text)
        results

        time.sleep(120)

        creation_id = results['id']
        second_url = 'https://graph.facebook.com/v15.0/{}/media_publish'.format(ig_user_id)
        second_payload = {'creation_id':creation_id, 'access_token': access_token}

        r2 = requests.post(second_url, data= second_payload)
        print(r2.text)

        return r2
    
    def delete_media(self):
        
        
        #List objects in bucket for delete 
        for obj in s3.Bucket(bucket_name).objects.all():
            obj = obj
            #print(obj)
            obj.delete()
            
        
        #os.remove(self.name_media)
        
        #copy video to other folder and delete from root folder
        source_media = self.source_videos+'/{}'.format(self.name_media)
        destiny =self.source_videos+'/ya_public_insta/{}'.format(self.name_media)
        shutil.copy(source_media, destiny)
        os.system('wmic process where name="ffmpeg-win64-v4.2.2.exe" delete')
        time.sleep(120)
        os.remove(source_media)

### Run process until video is upload

In [ ]:
insta_status = 'No'

In [ ]:
while insta_status == 'No':
    upload = upload_videos()
    upload_instagram = upload.upload_instagram()
    upload1 = json.loads(upload_instagram.text)
    try:
        print(upload1['id'])
        insta_status = 'Ok'
        
        #Delete object from amazon s3 and windows
        upload.delete_media()
    except:
        print(upload1['error'])
        insta_status = 'No'
        #Delete object from amazon s3 and windows
        upload.delete_media()